In [258]:
import numpy as np
import pandas as pd
from pathlib import Path

In [259]:
def build_schedule(solved_per_problem, evals_per_problem, max_iters = 100, log = True):
    assert(len(solved_per_problem) == len(evals_per_problem))

    prior_likelihood = np.ones(len(solved_per_problem)) / len(solved_per_problem)
    expected_evals = np.zeros(len(solved_per_problem))
    
    schedule = []

    for _ in range(0, max_iters):
        # solve_per_eval = (solved_per_problem.mul(prior_likelihood, axis=0).mean(axis=0) / evals_per_problem.mul(prior_likelihood, axis=0).mean(axis=0))
        solve_per_eval = (solved_per_problem / evals_per_problem).mul(prior_likelihood, axis=0).mean(axis=0)
        idxmax = solve_per_eval.idxmax()

        schedule.append(idxmax)
        expected_evals += evals_per_problem[idxmax].mul(prior_likelihood, axis=0).values

        prior_likelihood = (1 - solved_per_problem[idxmax]).values * prior_likelihood
        prior_likelihood = prior_likelihood / sum(prior_likelihood)

        if log: print(f"choose: {idxmax}, unsolved: {prior_likelihood.sum():.2f}, expected_evals: {expected_evals.mean():.2f}")
    
    return (schedule, prior_likelihood, expected_evals)

In [260]:
# solved_per_problem = pd.DataFrame([[0.5, 0.1], [0.1, 0.5]], index = ["A", "B"]).transpose()
# evals_per_problem = pd.DataFrame([[100, 10], [10, 100]], index = ["A", "B"]).transpose()

# build_schedule(solved_per_problem, evals_per_problem, max_iters=10, log = True)

In [261]:
def evaluate_schedule(solved_per_problem, evals_per_problem, schedule, penalty):
    assert(len(solved_per_problem) == len(evals_per_problem))

    prior_likelihood = np.ones(len(solved_per_problem))
    expected_evals = np.zeros(len(solved_per_problem))

    # setup evaluations per algorithm
    evals_per_algo = {}
    for algo in schedule:
        if algo not in evals_per_algo:
            evals_per_algo[algo] = 0
    
    # setup expected evaluations per restart
    evals_per_restart = []

    for algo in schedule:
        evals_this_run = evals_per_problem[algo].mul(prior_likelihood, axis=0).values
        evals_per_algo[algo] += evals_this_run.mean()
        evals_per_restart.append(evals_this_run)
        
        expected_evals += evals_this_run
        prior_likelihood = (1 - solved_per_problem[algo]).values * prior_likelihood

    ert = pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]

    return (ert, evals_per_algo, evals_per_restart)

In [301]:
dim = 10
files = Path(f"output/{dim}D").glob("*/*.csv")

dfs = []

for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

In [302]:
all_df = pd.concat(dfs, ignore_index=True)

# Just CMA-ES
# all_df = all_df[all_df["param_id"].isin(["default", "2L", "4L", "8L", "16L", "32L", "64L"])]

all_df.head()

,log10_target,solved,evaluations,fid,iid,rep,dim,param_id
0,2.0,True,375,6,101,1,10,Powell
1,1.8,True,375,6,101,1,10,Powell
2,1.6,True,479,6,101,1,10,Powell
3,1.4,True,487,6,101,1,10,Powell
4,1.2,True,487,6,101,1,10,Powell


In [303]:
all_df.iid.max()

200

In [304]:
mean_evals = all_df.groupby(["log10_target", "fid", "param_id"], as_index = False).agg({ # , "iid"
    "evaluations": "mean", 
    "solved": "mean",
})

In [305]:
# all_df.groupby(["log10_target", "fid", "param_id", "solved"], as_index = False).agg({ # , "iid"
#     "evaluations": "mean",
# }).query("log10_target == -8.0 and param_id == 'Powell'").sort_values(["fid", "param_id", "solved"])

In [306]:
# mean_evals["spe"] = mean_evals["solved"] / mean_evals["evaluations"]
mean_evals[np.logical_and(mean_evals["fid"] == 24, mean_evals["log10_target"] == -8.0)]

,log10_target,fid,param_id,evaluations,solved
230,-8.0,24,16L,65633.60,0.0
231,-8.0,24,2L,13463.00,0.0
232,-8.0,24,32L,98476.80,0.0
233,-8.0,24,4L,24163.60,0.0
234,-8.0,24,64L,165696.00,0.0
235,-8.0,24,8L,41936.80,0.0
236,-8.0,24,L-BFGS-B,435.60,0.0
237,-8.0,24,Powell,1451.70,0.0
238,-8.0,24,SLSQP,305.94,0.0
239,-8.0,24,default,8530.20,0.0


In [307]:
# mean_evals["problem_id"] = mean_evals.apply(lambda x: f"f{x.fid:.0f}-i{x.iid:.0f}-d5-t{x.log10_target}", axis=1)
mean_evals["problem_id"] = mean_evals.apply(lambda x: f"f{x.fid:02}-d{dim:02}-t{x.log10_target:.1f}", axis=1)
mean_evals.head()

,log10_target,fid,param_id,evaluations,solved,problem_id
0,-8.0,1,16L,10582.50,1.0,f01-d10-t-8.0
1,-8.0,1,2L,2240.55,1.0,f01-d10-t-8.0
2,-8.0,1,32L,14654.32,1.0,f01-d10-t-8.0
3,-8.0,1,4L,3613.21,1.0,f01-d10-t-8.0
4,-8.0,1,64L,21028.27,1.0,f01-d10-t-8.0


In [308]:
mean_evals = mean_evals.drop(columns = ["log10_target", "fid"]) #, "iid"
mean_evals.head()

,param_id,evaluations,solved,problem_id
0,16L,10582.50,1.0,f01-d10-t-8.0
1,2L,2240.55,1.0,f01-d10-t-8.0
2,32L,14654.32,1.0,f01-d10-t-8.0
3,4L,3613.21,1.0,f01-d10-t-8.0
4,64L,21028.27,1.0,f01-d10-t-8.0


In [309]:
evals_per_problem = mean_evals.pivot(index = "problem_id", columns = "param_id", values = "evaluations")
evals_per_problem.head()

param_id,16L,2L,32L,4L,64L,8L,L-BFGS-B,Powell,SLSQP,default
problem_id,,,,,,,,,,
f01-d10-t-0.2,2521.69,515.55,3557.35,866.26,5331.37,1592.21,23.0,77.66,13.0,319.70
f01-d10-t-0.4,2750.57,557.96,3858.97,938.98,5741.95,1728.30,23.0,78.49,13.0,348.54
f01-d10-t-0.6,2937.56,605.11,4172.23,1019.26,6143.22,1852.47,23.0,78.99,13.0,379.46
f01-d10-t-0.8,3130.14,642.08,4438.38,1081.58,6562.56,1988.95,23.0,79.17,13.0,410.75
f01-d10-t-1.0,3336.29,696.07,4758.15,1150.85,6960.96,2119.12,23.0,79.39,13.0,443.14


In [310]:
solved_per_problem = mean_evals.pivot(index = "problem_id", columns = "param_id", values = "solved")
solved_per_problem.head()

param_id,16L,2L,32L,4L,64L,8L,L-BFGS-B,Powell,SLSQP,default
problem_id,,,,,,,,,,
f01-d10-t-0.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-0.4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-0.8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d10-t-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [311]:
print(solved_per_problem.loc[solved_per_problem.max(1) == 0].index.to_list())

['f04-d10-t-0.2', 'f04-d10-t-0.4', 'f04-d10-t-0.6', 'f04-d10-t-0.8', 'f04-d10-t-1.0', 'f04-d10-t-1.2', 'f04-d10-t-1.4', 'f04-d10-t-1.6', 'f04-d10-t-1.8', 'f04-d10-t-2.0', 'f04-d10-t-2.2', 'f04-d10-t-2.4', 'f04-d10-t-2.6', 'f04-d10-t-2.8', 'f04-d10-t-3.0', 'f04-d10-t-3.2', 'f04-d10-t-3.4', 'f04-d10-t-3.6', 'f04-d10-t-3.8', 'f04-d10-t-4.0', 'f04-d10-t-4.2', 'f04-d10-t-4.4', 'f04-d10-t-4.6', 'f04-d10-t-4.8', 'f04-d10-t-5.0', 'f04-d10-t-5.2', 'f04-d10-t-5.4', 'f04-d10-t-5.6', 'f04-d10-t-5.8', 'f04-d10-t-6.0', 'f04-d10-t-6.2', 'f04-d10-t-6.4', 'f04-d10-t-6.6', 'f04-d10-t-6.8', 'f04-d10-t-7.0', 'f04-d10-t-7.2', 'f04-d10-t-7.4', 'f04-d10-t-7.6', 'f04-d10-t-7.8', 'f04-d10-t-8.0', 'f04-d10-t0.0', 'f04-d10-t0.2', 'f04-d10-t0.4', 'f24-d10-t-0.4', 'f24-d10-t-0.6', 'f24-d10-t-0.8', 'f24-d10-t-1.0', 'f24-d10-t-1.2', 'f24-d10-t-1.4', 'f24-d10-t-1.6', 'f24-d10-t-1.8', 'f24-d10-t-2.0', 'f24-d10-t-2.2', 'f24-d10-t-2.4', 'f24-d10-t-2.6', 'f24-d10-t-2.8', 'f24-d10-t-3.0', 'f24-d10-t-3.2', 'f24-d10-t-3.4',

In [312]:
schedule, post_likelihood, expected_evals = build_schedule(solved_per_problem, evals_per_problem, max_iters = 1000)

choose: SLSQP, unsolved: 1.00, expected_evals: 0.32
choose: default, unsolved: 1.00, expected_evals: 4.94
choose: SLSQP, unsolved: 1.00, expected_evals: 5.26
choose: SLSQP, unsolved: 1.00, expected_evals: 5.58
choose: 4L, unsolved: 1.00, expected_evals: 17.92
choose: SLSQP, unsolved: 1.00, expected_evals: 18.24
choose: SLSQP, unsolved: 1.00, expected_evals: 18.57
choose: SLSQP, unsolved: 1.00, expected_evals: 18.90
choose: SLSQP, unsolved: 1.00, expected_evals: 19.23
choose: SLSQP, unsolved: 1.00, expected_evals: 19.57
choose: SLSQP, unsolved: 1.00, expected_evals: 19.90
choose: SLSQP, unsolved: 1.00, expected_evals: 20.23
choose: SLSQP, unsolved: 1.00, expected_evals: 20.57
choose: SLSQP, unsolved: 1.00, expected_evals: 20.90
choose: SLSQP, unsolved: 1.00, expected_evals: 21.24
choose: L-BFGS-B, unsolved: 1.00, expected_evals: 21.74
choose: L-BFGS-B, unsolved: 1.00, expected_evals: 22.23
choose: L-BFGS-B, unsolved: 1.00, expected_evals: 22.73
choose: 8L, unsolved: 1.00, expected_evals

In [313]:
f"{schedule}"

"['SLSQP', 'default', 'SLSQP', 'SLSQP', '4L', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'SLSQP', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '8L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '32L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', '16L', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-BFGS-B', 'L-B

In [255]:
pd.value_counts(schedule)

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_88438/2913986251.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(schedule)
/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_88438/2913986251.py:1: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(schedule)


Powell      361
2L          253
L-BFGS-B    167
64L         133
32L          68
SLSQP        13
16L           2
default       1
4L            1
8L            1
Name: count, dtype: int64

In [256]:
penalty = 1_000_000 * dim

(schedule_ert, epa, epr) = evaluate_schedule(solved_per_problem, evals_per_problem, schedule, penalty)
f"{schedule_ert:_.2f}"

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_88438/17078482.py:24: RuntimeWarning: divide by zero encountered in divide
  ert = pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


'939_455.42'

In [257]:
mean_evals.param_id.unique()

array(['16L', '2L', '32L', '4L', '64L', '8L', 'L-BFGS-B', 'Powell',
       'SLSQP', 'default'], dtype=object)

In [216]:
# ert_schedule = []

# for _ in range(0, 100):
#     best_schedule = []
#     best_perf = np.inf

#     for config in mean_evals.param_id.unique():
#         ext_schedule = ert_schedule.copy()
#         ext_schedule.extend([config])
#         perf = evaluate_schedule(solved_per_problem, evals_per_problem, ext_schedule, penalty)
#         if perf < best_perf:
#             best_perf = perf
#             best_schedule = ext_schedule
    
#     ert_schedule = best_schedule
    
#     print(f"{best_perf:_.2f}")

In [185]:
# ert_schedule

In [186]:
# sbs
all_ert = (evals_per_problem / solved_per_problem).clip(upper=penalty).mean()
all_ert.sort_values()

param_id
32L         1.031962e+06
16L         1.307927e+06
64L         1.730237e+06
8L          1.977982e+06
4L          1.990959e+06
2L          2.357224e+06
default     3.008707e+06
L-BFGS-B    4.672970e+06
SLSQP       4.846449e+06
Powell      5.818368e+06
dtype: float64

In [187]:
sbs = min(all_ert)
f"{sbs:_.2f}"

'1_031_961.75'

In [188]:
# vbs
vbs = (evals_per_problem / solved_per_problem).clip(upper=penalty).apply(min, axis = 1).mean()
f"{vbs:_.2f}"

'841_007.04'

In [189]:
print(f"SBS-VBS Gap closed (Resubstitution): {100 - 100 * (schedule_ert - vbs) / (sbs - vbs):.2f}%")

SBS-VBS Gap closed (Resubstitution): 48.44%


In [190]:
penalty = 1_000_000 * dim

erts = []

# method = "sbs"
method = "normal"

for i in range(1, 25):
    print(f"Evaluating f{i:02}", end = "\r")
    filter_string = f"f{i:02}-"

    just_fi = [x for x in solved_per_problem.index if x.startswith(filter_string)]
    all_but_fi = [x for x in solved_per_problem.index if not x.startswith(filter_string)]

    if method == "sbs":
        all_ert = (evals_per_problem.loc[all_but_fi] / solved_per_problem.loc[all_but_fi]).clip(upper=penalty).mean()
        schedule = [all_ert.sort_values().index[0]]
        print(schedule)
    else:
        schedule, post_likelihood, expected_evals = build_schedule(solved_per_problem.loc[all_but_fi], evals_per_problem.loc[all_but_fi], max_iters = 1000, log = False)
    
    (ert, epa, epr) = evaluate_schedule(solved_per_problem.loc[just_fi], evals_per_problem.loc[just_fi], schedule, penalty)
    erts.append(ert)

lopo_ert = np.mean(erts)
f"{lopo_ert:_.2f}"

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_88438/3395898693.py:11: RuntimeWarning: divide by zero encountered in divide
  return pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_88438/3395898693.py:11: RuntimeWarning: divide by zero encountered in divide
  return pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


'999_246.10'

In [191]:
schedule

['SLSQP',
 'default',
 'SLSQP',
 'SLSQP',
 '4L',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 '8L',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 '32L',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 '16L',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-

In [192]:
# erts_sbs = erts

In [193]:
((pd.DataFrame(erts) - pd.DataFrame(erts_sbs)) / pd.DataFrame(erts_sbs)) # .sum()

,0
0,-0.992881
1,-0.608039
2,21.696485
3,1.085033
4,-0.862615
5,-0.363800
6,1.690085
7,-0.473369
8,-0.770691
9,-0.490247


In [194]:
np.log10(erts).mean()

4.3248571293810985

In [195]:
print(f"SBS-VBS Gap closed (LOPO): {100 - 100 * (lopo_ert - vbs) / (sbs - vbs):.2f}%")

SBS-VBS Gap closed (LOPO): 17.13%


In [141]:
pd.DataFrame(schedule).value_counts()

0       
Powell      841
L-BFGS-B    111
32L          20
4L           16
SLSQP        10
16L           1
default       1
Name: count, dtype: int64

In [142]:
(solved_per_problem["Powell"] > 0).mean()

0.5555555555555556

In [98]:
((solved_per_problem / evals_per_problem)["32L"] > 0).sort_values()

problem_id
f4-d10-t-8.0     False
f24-d10-t-6.0    False
f24-d10-t-6.2    False
f24-d10-t-6.4    False
f24-d10-t-6.6    False
                 ...  
f16-d10-t1.6      True
f16-d10-t1.4      True
f16-d10-t1.2      True
f17-d10-t-0.6     True
f9-d10-t2.0       True
Name: 32L, Length: 1224, dtype: bool

In [99]:
(solved_per_problem / evals_per_problem)["32L"]

problem_id
f1-d10-t-0.2    0.000281
f1-d10-t-0.4    0.000260
f1-d10-t-0.6    0.000241
f1-d10-t-0.8    0.000226
f1-d10-t-1.0    0.000212
                  ...   
f9-d10-t1.2     0.000181
f9-d10-t1.4     0.000208
f9-d10-t1.6     0.000234
f9-d10-t1.8     0.000260
f9-d10-t2.0     0.000292
Name: 32L, Length: 1224, dtype: float64